In [24]:
import numpy as np
import matplotlib.pyplot as plt

## SLIP  
library to calculate Spring Loaded Inverted Pendulum

In [25]:
# defined params
RIGHT = 0
LEFT = 1

In [26]:
# mechanical params
g = 9.81 # m/s^2
m = 1.2 # kg

In [49]:
# robot initial settings
z0 = 158 * 0.001
z_flight = 0.04
foot_dist_y_base = 0.03
foot_dist_x_max = 0.12

T_sup_base = 0.6
T_sup_min = 0.3

Tc = np.sqrt(z0 / g)

vy_max = 0.05

In [50]:
# basic state calculation
def calc_basic_passing_com_vel(p, T_sup):
    return (p/Tc) * ( np.sinh(T_sup/Tc) / ( 1 - np.cosh(T_sup/Tc)))    
def calc_basic_unpassing_com_vel(p, T_sup):
    return (p/Tc) * ( np.sinh(T_sup/Tc) / ( -1 - np.cosh(T_sup/Tc)))

def calc_basic_passing_foot_pos(v, T_sup):
    return Tc*v * (np.cosh(T_sup/Tc)-1) / np.sinh(T_sup/Tc)
def calc_basic_unpassing_foot_pos(v, T_sup):
    return Tc*v * (np.cosh(T_sup/Tc)+1) / np.sinh(T_sup/Tc)

In [51]:
# calculate robot initial settings
v_mag_boarder = calc_basic_passing_com_vel(foot_dist_x_max, T_sup_base)
v_mag_max = calc_basic_passing_com_vel(foot_dist_x_max, T_sup_min)

vx_max = v_mag_max

In [52]:
# order normalization
def normalize_vel(v):
    v_mag = np.sqrt(v[0]**2 + v[1]**2)
    if v_mag > 0.34:
        return [0.34/v_mag*v[0], 0.34/v_mag*v[1], v[2]]
    else:
        return v
    
def calc_T_sup(v_mag):
    if v_mag == 0:
        return 0
    elif v_mag < 0.25:
        return 0.6
    else:
        return 0.6 - 0.3 * ( (v_mag - 0.25) / (0.34 - 0.25) )

In [53]:
# rotation
def calc_rot_angle_limit(cvn_last):
    # return 0 for too small vel
    if abs(cvn_last[0]) < 0.005 or abs(cvn_last[1]) < 0.005:
        return [0,0]

    a1 = np.arctan2(cvn_last[1], cvn_last[0])
    y_v_lim = 0.03 / Tc
    cv_mag = np.sqrt(cvn_last[0]**2 + cvn_last[1]**2)
    if cv_mag < y_v_lim: # if no concerns to pass the pivot
        a2 = 0.5
    else:
        a2 = np.arccos(y_v_lim / np.sqrt(cvn_last[0]**2 + cvn_last[1]**2))
    a2 = a2 * -1 * (a1/abs(a1))

    # reduce angle limits
    a1*=0.3
    a2*=0.3

    angle_limits = []
    if a1 > a2:
        angle_limits = [a1, a2]
    else:
        angle_limits = [a2, a1]
    # print("angle_limits:", angle_limits)
    return angle_limits

def rotate_vec(vec, angle):
    c = np.cos(angle)
    s = np.sin(angle)
    R = np.array([[c, -s],
                  [s,  c]])
    vec_rot = R @ np.array(vec)
    return vec_rot.tolist()

In [54]:
kp = 0.01
kd = 0.001

# foot position pd
def foot_pos_pd(cvn_last, cvn_m1_last, vd, pn, T_sup):
    err = [0,0]
    err[0] = vd[0] - cvn_last[0]

    basic_foot_pos = foot_dist_y_base * -pn[1]/abs(pn[1])
    vd_y_sum = calc_basic_unpassing_com_vel(basic_foot_pos, T_sup) + vd[1]
    err[1] = vd_y_sum - cvn_last[1]

    derr = [0,0]
    derr[0] = (-cvn_last[0] + cvn_m1_last[0])
    derr[1] = (-cvn_last[1] + cvn_m1_last[1])

    p_base = [0,0]
    p_base[0] = calc_basic_passing_foot_pos(cvn_last[0], T_sup)
    p_base[1] = calc_basic_unpassing_foot_pos(cvn_last[1], T_sup)

    px = p_base[0] - kp * err[0] - kd * derr[0]
    py = p_base[1] - kp * err[1] - kd * derr[1]

    return [px, py]


In [55]:
# LIP calculation
def calc_LIP_p(t, cp0, cv0):
    x_t = cp0[0] * np.cosh(t/Tc) + Tc * cv0[0] * np.sinh(t/Tc)
    y_t = cp0[1] * np.cosh(t/Tc) + Tc * cv0[1] * np.sinh(t/Tc)
    return [x_t, y_t]

def calc_LIP_v(t, cp0, cv0):
    vx_t = cp0[0] / Tc * np.sinh(t/Tc) + cv0[0] * np.cosh(t/Tc)
    vy_t = cp0[1] / Tc * np.sinh(t/Tc) + cv0[1] * np.cosh(t/Tc)
    return [vx_t, vy_t]

In [56]:
# double support sprine
def calc_double_support_sprine_coeff(T_ds, cpn_d_0, cvn_d_0, can_d_0, cpn_d_T, cvn_d_T, can_d_T):
    a0 = cpn_d_0
    a1 = cvn_d_0
    a2_x = 1/2*can_d_0[0]
    a2_y = 1/2*can_d_0[1]

    a3_x = 1/(2*T_ds**3) * ( 20*(cpn_d_T[0]-cpn_d_0[0]) - (8*cvn_d_T[0] + 12*cvn_d_0[0])*T_ds - (3*can_d_0[0] - can_d_T[0])*T_ds**2 )
    a3_y = 1/(2*T_ds**3) * ( 20*(cpn_d_T[1]-cpn_d_0[1]) - (8*cvn_d_T[1] + 12*cvn_d_0[1])*T_ds - (3*can_d_0[1] - can_d_T[1])*T_ds**2 )

    a4_x = 1/(2*T_ds**4) * ( 30*(cpn_d_0[0]-cpn_d_T[0]) + (14*cvn_d_T[0] + 16*cvn_d_0[0])*T_ds + (3*can_d_0[0] - 2*can_d_T[0])*T_ds**2 )
    a4_y = 1/(2*T_ds**4) * ( 30*(cpn_d_0[1]-cpn_d_T[1]) + (14*cvn_d_T[1] + 16*cvn_d_0[1])*T_ds + (3*can_d_0[1] - 2*can_d_T[1])*T_ds**2 )

    a5_x = 1/(2*T_ds**5) * ( 12*(cpn_d_T[0]-cpn_d_0[0]) - 6*(cvn_d_T[0] + cvn_d_0[0])*T_ds - (can_d_0[0] - can_d_T[0])*T_ds**2 )
    a5_y = 1/(2*T_ds**5) * ( 12*(cpn_d_T[1]-cpn_d_0[1]) - 6*(cvn_d_T[1] + cvn_d_0[1])*T_ds - (can_d_0[1] - can_d_T[1])*T_ds**2 )

    ax = [a0[0], a1[0], a2_x, a3_x, a4_x, a5_x]
    ay = [a0[1], a1[1], a2_y, a3_y, a4_y, a5_y]

    return [ax, ay]

def calc_double_support_com_p(t, coeff):
    x_t = coeff[0][0] + coeff[0][1]*t + coeff[0][2]*t**2 + coeff[0][3]*t**3 + coeff[0][4]*t**4 + coeff[0][5]*t**5
    y_t = coeff[1][0] + coeff[1][1]*t + coeff[1][2]*t**2 + coeff[1][3]*t**3 + coeff[1][4]*t**4 + coeff[1][5]*t**5

    return [x_t, y_t]

In [57]:
def calc_z_height(t, T_sup, double_support_ratio):
    T_ss = T_sup * (1-double_support_ratio)

    z = z0 + np.cos(np.pi * t / T_ss) * T_sup/20 - T_sup/4

    if t < T_ss:
        zh = np.sin(np.pi * t / T_ss) * z_flight
        zf = z - zh
    else:
        zh = np.cos(2*np.pi * (t - T_ss/2) / T_ss) * z_flight/2 + z_flight/2
        zf = z - zh

    return [z,zf]

## main code : Gait Generation

In [58]:
# mode
WAIT = 1
WALK = 2

# phase
START = 1
END = 2
SINGLE = 3
DOUBLE = 4
FLIGHT = 5

In [59]:
# control state variables
# mode/phase
mode = 0
mode_next = 0
phase = 0
phase_next = 0

# counts
phase_length = 0
phase_count = 0

# pivot
RIGHT = 1
LEFT = -1
pivot = 0



In [60]:
# SLIP state variables
# com state at last step
cpn_last = [0,0]
cvn_last = [0,0]
cvn_m1_last = [0,0]

# foot step
pn = [0,0]
pn_p1 = [0,0]
p_n2p1 = [0,0,0]
p_n2m1 = [0,0,0]
pivot = 0

# double support calculation
T_ds = 0
ds_ratio = 0.2
ds_coeff = [[0,0,0,0,0,0],[0,0,0,0,0,0]]
cpn_d_0 = [0,0]
cvn_d_0 = [0,0]
can_d_0 = [0,0]
cpn_d_T = [0,0]
cvn_d_T = [0,0]
can_d_T = [0,0]

# supporting time
T_sup = 0
T_sup_last = 0

# angle
body_angle = 0

In [61]:
# initialize
mode = WAIT
phase = START
pivot = RIGHT
cpn_last = [0, -1 * pivot * foot_dist_y_base]
pn = [0, pivot * foot_dist_y_base]
p_n2m1 = [0, -2 * pivot * foot_dist_y_base, 0]
T_sup_last = 0

In [ ]:
def init_phase(a,b,c):
    global mode
    global phase
    global phase_count 
    global phase_length

    mode = a
    phase = b
    phase_count = 0
    phase_length = c * 100

def initialize_state_variables():
    global cpn_last
    global cvn_last
    global cvn_m1_last
    global pn
    global p_n2m1
    global cpn_d_0
    global cvn_d_0
    global can_d_0

    pn = pn_p1
    p_n2m1 = [-p_n2p1[0], -p_n2p1[1], -p_n2p1[2]]
    cvn_m1_last = cvn_last
    cpn_last = calc_LIP_p(T_sup, [-pn[0], -pn[1]], cvn_last)
    cvn_last = calc_LIP_v(T_sup, [-pn[0], -pn[1]], cvn_last)

    cpn_d_0 = calc_LIP_p(T_sup*(1 - ds_ratio/2), [-pn[0], -pn[1]], cvn_last)
    cpn_d_0 = [cpn_d_0[0] - cpn_last[0], cpn_d_0[1] - cpn_last[1]]
    cvn_d_0 = calc_LIP_v(T_sup*(1 - ds_ratio/2), [-pn[0], -pn[1]], cvn_last)
    can_d_0 = [cpn_d_0[0] / Tc**2, cpn_d_0[1] / Tc**2]

def update_state_variables(vd):
    global pn_p1
    global p_n2p1
    global T_ds
    global ds_coeff
    global cpn_d_0
    global cvn_d_0
    global can_d_0
    global cpn_d_T
    global cvn_d_T
    global can_d_T
    global T_sup
    global T_sup_last
    global body_angle

    # normalize control input
    vd = normalize_vel(vd)
    v_mag = np.sqrt(vd[0]**2 + vd[1]**2)
    T_sup_last = T_sup
    T_sup = calc_T_sup(v_mag)


    # decide angle
    angle_limits = calc_rot_angle_limit(cvn_last)
    rotation = vd[2]
    if rotation > 0:
        rotation = angle_limits[0] * abs(rotation)
    else:
        rotation = angle_limits[1] * abs(rotation)
    body_angle = rotation
    # rotate vel
    cvn_last_rot = rotate_vec(cvn_last, rotation)


    # decide pn_p1
    pn_p1 = foot_pos_pd(cvn_last_rot, cvn_m1_last, vd, pn, T_sup)
    p_n2p1 = [cpn_last[0]+pn_p1[0], cpn_last[1]+pn_p1[1], body_angle]

    # calculate double support trajectory
    # convert com information at the start of double support duration
    cpn_d_0 = rotate_vec(cpn_d_0, -rotation)
    cpn_d_0 = [cpn_d_0[0] - pn_p1[0], cpn_d_0[1] - pn_p1[1]]
    cvn_d_0 = rotate_vec(cvn_d_0, -rotation)
    can_d_0 = rotate_vec(can_d_0, -rotation)

    cpn_d_T = calc_LIP_p(T_sup*ds_ratio/2, [-pn_p1[0], -pn_p1[1]], cvn_last_rot)
    cvn_d_T = calc_LIP_v(T_sup*ds_ratio/2, [-pn_p1[0], -pn_p1[1]], cvn_last_rot)
    can_d_T = [cpn_d_T[0] / Tc**2, cpn_d_T[1] / Tc**2]

    # calculate coeff
    T_ds = (T_sup + T_sup_last) * ds_ratio/2
    ds_coeff = calc_double_support_sprine_coeff(T_ds, cpn_d_0, cvn_d_0, can_d_0, cpn_d_T, cvn_d_T, can_d_T)


In [ ]:
# start walking
mode = WALK

vd = [0.2, 0.0, 0.0]  # desired vel x,y,rotation
com_right = []
com_left = []

for i in range(10*60):
    if phase == START:
        if phase_count == 0:
            # calculate state variables
            update_state_variables(vd)
            phase_length = T_ds * 100
        
        # calculate com
        com = calc_double_support_com_p(phase_count/100, ds_coeff)
        com_z = z0

        # send order
        com_order_right = []
        com_order_left = []

        if pivot == RIGHT:
            com_order_right = [com[0], com[1], com_z, 0,0]
            com_order_left = [p_n2m1[0]+com[0], p_n2m1[1]+com[1], com_z, 0,0]
        else:
            com_order_right = [p_n2m1[0]+com[0], p_n2m1[1]+com[1], com_z, 0,0]
            com_order_left = [com[0], com[1], com_z, 0,0]

        phase_count += 1
        if phase_count == phase_length: 
            init_phase(WALK, SINGLE, T_sup * (1-ds_ratio) * 100)

    elif phase == END:
        phase_count += 1
        if phase_count == phase_length:
            init_phase(WAIT, START, 0)

    elif phase == SINGLE:
        com = calc_LIP_p(phase_count/100 + T_sup*ds_ratio/2, [-pn[0], -pn[1]], cvn_last)

        if phase_count == int(phase_length/2):
            update_state_variables(vd)
            if vd[0] == 0 and vd[1] == 0:
                phase_next = END
            else:
                phase_next = DOUBLE

        elif phase_count > int(phase_length/2):
            pass
        else:
            pass
        pass

        phase_count += 1
        if phase_count == phase_length:
            init_phase(WALK, phase_next, T_ds * 100)

    elif phase == DOUBLE:
        if phase_count == 0:
            pivot *= -1
            initialize_state_variables()


        phase_count += 1
        if phase_count == phase_length:
            init_phase(WALK, SINGLE, T_sup * (1-ds_ratio) * 100)